In [1]:
from IPython.display import clear_output
import requests as rq
import pandas as pd
import os
import pubchempy as pcp
import re

In [2]:
df_keys = ['cid','name']
df = pd.DataFrame(columns = df_keys)
df

,cid,name


In [3]:
df_dic = {}

In [4]:
def get_cid(cid):
    return cid

In [5]:
a = get_cid(1983)
a

1983

In [24]:
def get_name(response):
    return response['Record']['RecordTitle']

In [7]:
def get_atc_code(response):
    pat = r"\"[A-Z]\d{2}[A-Z]{2}\d{2}\""
    atc_code_found = re.search(pat, response.text)
    if atc_code_found:
        return atc_code_found.group(0).strip('"')

In [31]:
cid = 1980
url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON'
response = rq.get(url)
get_atc_code(response)


In [36]:
response_json = rq.get(url).json()
get_name(response_json)

{'RecordType': 'CID', 'RecordNumber': 1980, 'RecordTitle': '3,6-Diacetoxy-3,3a,6,6a-tetrahydrofuro[3,2-b]furan-2,5-dione', 'Section': [{'TOCHeading': 'Structures', 'Description': 'Structure depictions of this compound, including computationally generated two-dimensional (2D) and three-dimensional (3D) structures, as well as experimentally determined 3D single-crystal structures.', 'Section': [{'TOCHeading': '2D Structure', 'Description': 'A two-dimensional (2D) structure representation of the compound.  Because this structure is processed through chemical structure standardization (Hähnke et al., J. Cheminform. 2018, 10, 36), it is not necessarily the same as the structures provided by individual data contributors.  ', 'URL': 'https://doi.org/10.1186/s13321-018-0293-8', 'DisplayControls': {'MoveToTop': True}, 'Information': [{'ReferenceNumber': 4, 'Value': {'Boolean': [True]}}]}, {'TOCHeading': '3D Conformer', 'Description': 'A three-dimensional (3D) structure representation of the com

'3,6-Diacetoxy-3,3a,6,6a-tetrahydrofuro[3,2-b]furan-2,5-dione'

In [53]:
for partition in response.text.partition('"RecordTitle": '):
    print('part', partition.split(',"Section":'))


part ['{\n  "Record": {\n    "RecordType": "CID",\n    "RecordNumber": 1980,\n    ']
part ['"RecordTitle": ']
part ['"3,6-Diacetoxy-3,3a,6,6a-tetrahydrofuro[3,2-b]furan-2,5-dione",\n    "Section": [\n      {\n        "TOCHeading": "Structures",\n        "Description": "Structure depictions of this compound, including computationally generated two-dimensional (2D) and three-dimensional (3D) structures, as well as experimentally determined 3D single-crystal structures.",\n        "Section": [\n          {\n            "TOCHeading": "2D Structure",\n            "Description": "A two-dimensional (2D) structure representation of the compound.  Because this structure is processed through chemical structure standardization (Hähnke et al., J. Cheminform. 2018, 10, 36), it is not necessarily the same as the structures provided by individual data contributors.  ",\n            "URL": "https://doi.org/10.1186/s13321-018-0293-8",\n            "DisplayControls": {\n              "MoveToTop": true\n

In [73]:
response.text.partition('"RecordTitle": ')[2].split(',\n')[0]

'"3,6-Diacetoxy-3,3a,6,6a-tetrahydrofuro[3,2-b]furan-2,5-dione"'

In [81]:
print(response.text.split('RecordTitle": ')[1].split('"')[1])

3,6-Diacetoxy-3,3a,6,6a-tetrahydrofuro[3,2-b]furan-2,5-dione


In [89]:
df_atc = pd.read_csv('pubchem_100k_200k_atc.csv', index_col=False)
df_atc.info()
df_atc.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cid       1 non-null      int64 
 1   name      1 non-null      object
 2   atc_code  1 non-null      object
dtypes: int64(1), object(2)
memory usage: 156.0+ bytes


,cid,name,atc_code
0,219078,Lacosamide,N03AX18


In [90]:
df_no_atc = pd.read_csv('pubchem_100k_200k_no_atc.csv', index_col=False)
df_no_atc.info()
df_no_atc.tail(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cid       0 non-null      object
 1   name      0 non-null      object
 2   atc_code  0 non-null      object
dtypes: object(3)
memory usage: 132.0+ bytes


,cid,name,atc_code


In [14]:
df_keys = ['cid', 'name', 'atc_code']

In [15]:
import random

In [101]:
cids = list(random.sample(range(100000,168500000),100000))
df_atc = pd.read_csv('pubchem_ran_atc.csv')
df_no_atc = pd.read_csv('pubchem_ran_no_atc.csv')
df_dic = {}

i=0
for cid in cids:
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON'
    
    response_text = rq.get(url)
    response_json = rq.get(url).json()
    
    
    try:
        df_dic['cid'] = get_cid(cid)
        df_dic['name'] = get_name(response_json)
        df_dic['atc_code'] = get_atc_code(response_text)

        if response_text:
            if get_atc_code(response_text):
                df_atc.loc[len(df_atc)] = df_dic
            else:
                df_no_atc.loc[len(df_no_atc)] = df_dic
            df_atc.to_csv('pubchem_ran_atc.csv', index=False)
            df_no_atc.to_csv('pubchem_ran_no_atc.csv', index=False)
    except:
        pass

    i += 1
    clear_output()
    print(i, cid)

display(df_atc.head())
display(df_no_atc.tail(10))

{'RecordType': 'CID', 'RecordNumber': 8570259, 'RecordTitle': 'N-[(5-chlorothiophen-2-yl)methyl]-N-ethyl-2-(6-ethylthieno[2,3-d]pyrimidin-4-yl)sulfanylacetamide', 'Section': [{'TOCHeading': 'Structures', 'Description': 'Structure depictions of this compound, including computationally generated two-dimensional (2D) and three-dimensional (3D) structures, as well as experimentally determined 3D single-crystal structures.', 'Section': [{'TOCHeading': '2D Structure', 'Description': 'A two-dimensional (2D) structure representation of the compound.  Because this structure is processed through chemical structure standardization (Hähnke et al., J. Cheminform. 2018, 10, 36), it is not necessarily the same as the structures provided by individual data contributors.  ', 'URL': 'https://doi.org/10.1186/s13321-018-0293-8', 'DisplayControls': {'MoveToTop': True}, 'Information': [{'ReferenceNumber': 1, 'Value': {'Boolean': [True]}}]}, {'TOCHeading': '3D Conformer', 'Description': 'A three-dimensional 

KeyboardInterrupt: 

In [16]:
cids = list(range(1980, 1990))
for cid in cids:
    print(cid)
    
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON'
    response = rq.get(url).json()
    for head in response['Record']['Section']:
        if head['TOCHeading'] == 'Pharmacology and Biochemistry':
            for key in head['Section']:
                #print(key)
                if key['TOCHeading'] == 'ATC Code':
                    for prop in key['Information']:
                        if 'StringWithMarkup' in prop['Value'].keys() :
                            print(prop['Value']['StringWithMarkup'][0]['String'])
                       


1980
1981
M01AB11
M - Musculo-skeletal system
1982
1983
N02BE01
N - Nervous system
1984
1985
G - Genito urinary system and sex hormones
P - Antiparasitic products, insecticides and repellents
A - Alimentary tract and metabolism
1986
S01EC01
S - Sensory organs
1987
1988
1989
A - Alimentary tract and metabolism


In [17]:
cids = list(range(1980, 1990))
for cid in cids:
    print(cid)
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/json'
    response = rq.get(url)
    pat = r"\"[A-Z]\d{2}[A-Z]{2}\d{2}\""
    atc_code_found = re.search(pat, response.text)
    if atc_code_found:
        print(atc_code_found.group(0).strip('"'))
    

1980
1981
M01AB11
1982
1983
N02BE01
1984
1985
G01AB01
1986
S01EC01
1987
1988
1989
A10BB31


In [18]:
cid = 1983
url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/json'
response = rq.get(url)
pat = r"\"[A-Z]\d{2}[A-Z]{2}\d{2}\""
atc_code =  re.search(pat, response.text)
print(atc_code.group(0).strip('"'))


N02BE01


In [20]:
for head in response['Record']['Section']:
    if head['TOCHeading'] == 'Pharmacology and Biochemistry':
        for key in head['Section']:
            #print(key)
            if key['TOCHeading'] == 'ATC Code':
                for prop in key['Information']:
                    print(prop)
                    

TypeError: 'Response' object is not subscriptable

In [ ]:
drugbank_df = pd.read_csv('..\Drugbank database\drugbank_all_full_database.xml\molecule_dataframe.csv')
drugbank_df.columns


Index(['Unnamed: 0', '@type', '@created', '@updated', 'drugbank-id', 'name',
       'cas-number', 'unii', 'average-mass', 'monoisotopic-mass', 'groups',
       'mixtures', 'manufacturers', 'experimental-properties', 'snp-effects',
       'snp-adverse-drug-reactions', 'targets', 'enzymes', 'carriers',
       'transporters', 'logP', 'logS', 'Water Solubility', 'IUPAC Name',
       'Traditional IUPAC Name', 'Molecular Weight', 'Monoisotopic Weight',
       'SMILES', 'Molecular Formula', 'InChI', 'InChIKey',
       'Polar Surface Area (PSA)', 'Refractivity', 'Polarizability',
       'Rotatable Bond Count', 'H Bond Acceptor Count', 'H Bond Donor Count',
       'pKa (strongest acidic)', 'Physiological Charge', 'Number of Rings',
       'Bioavailability', 'Rule of Five', 'Ghose Filter', 'MDDR-Like Rule',
       'atc_code', 'atc_code_0', 'atc_code_0_$', 'atc_code_1', 'atc_code_1_$',
       'atc_code_2', 'atc_code_2_$', 'atc_code_3', 'atc_code_3_$'],
      dtype='object')

In [145]:
comp = pcp.get_compounds(1983)

In [146]:
comp[0]

Compound(1983)

In [36]:
atc_codes=pd.read_csv('atc_codes.csv', sep=';')
atc_dict = atc_codes.to_dict()
atc_dict

{'atc_code': {0: 'A',
  1: 'B',
  2: 'C',
  3: 'D',
  4: 'G',
  5: 'H',
  6: 'J',
  7: 'L',
  8: 'M',
  9: 'N',
  10: 'P',
  11: 'R',
  12: 'S',
  13: 'V'},
 'atc_desc': {0: 'ALIMENTARY TRACT AND METABOLISM',
  1: 'BLOOD AND BLOOD FORMING ORGANS',
  2: 'CARDIOVASCULAR SYSTEM',
  3: 'DERMATOLOGICALS',
  4: 'GENITO URINARY SYSTEM AND SEX HORMONES',
  5: 'SYSTEMIC HORMONAL PREPARATIONS, EXCL. SEX HORMONES AND INSULINS',
  6: 'ANTIINFECTIVES FOR SYSTEMIC USE',
  7: 'ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS',
  8: 'MUSCULO-SKELETAL SYSTEM',
  9: 'NERVOUS SYSTEM',
  10: 'ANTIPARASITIC PRODUCTS, INSECTICIDES AND REPELLENTS',
  11: 'RESPIRATORY SYSTEM',
  12: 'SENSORY ORGANS',
  13: 'VARIOUS'}}

In [25]:
pc_df = pd.read_csv('pubchem_1_2000_atc.csv')
pc_df['atc_code'] = pc_df['atc_code'].str[0]
pc_df['atc_desc'] = pc_df['atc_code'].apply(lambda x)
pc_df

,Unnamed: 0,cid,name,atc_code
0,0,1,Acetylcarnitine,N
1,1,137,Aminolevulinic acid,L
2,2,176,Acetic Acid,G
3,3,187,Acetylcholine,S
4,4,237,Quinacrine,P
...,...,...,...,...
67,67,1986,Acetazolamide,S
68,68,1989,Acetohexamide,A
69,69,1990,Acetohydroxamic acid,G
70,70,1993,Methacholine,V


In [14]:
import random
random.seed(77)
n = random.sample(range(0,168500000),10)
n


[67841280,
 87484765,
 52966160,
 64556986,
 51842987,
 30886865,
 78470395,
 127838834,
 149758725,
 164103470]

In [ ]:
168 500 000